In [ ]:
'''
execute this script in mae file folder
step1  create subfolder for xyz file
step2  in subfolder execute ff_gen(), generate gromacs files with user-defined name, need to define charge multiplicity and ... 
step3  calculate average charge of equivalent atoms as RESP charge and replace the charge in itp file generated by ff_gen()
        also solved the wrong 'MOL' resname bug in the itp file
'''
import os
import re
import linecache
import numpy as np
import veloxchem as vlx
import glob

def find_itp_file(path,grofile_name):
    itp_files = glob.glob(os.path.join(path, '**/'+grofile_name+'*.itp'), recursive=True)
    return os.path.basename(itp_files[0]) if itp_files else None

def mvff_files2subfolder(mae,grofile_name):
    subfolder = os.path.abspath ( '')+'/'+mae.strip(".01.mae")+'/' 
    ff_files = glob.glob(os.path.join(grofile_name+'*'), recursive=True)
    for f in ff_files:
        os.rename(f,subfolder+f)

def fileParse(INPUT,charge_column,atom_label_column):
    atom_label_column = int(atom_label_column)-1
    charge_column = int(charge_column)-1
    inputfile = str(INPUT)
    outputfile = inputfile.strip(".01.mae")
    print(inputfile,outputfile)
    newpath = os.path.abspath ( '')+'/'+str(outputfile)+'/'    # input file
    os.makedirs(newpath,exist_ok=True) 
    fp = open(inputfile,'r')
    number = []
    lineNumber = 1
    keyword = ":::"                                   #input('Keyword:')
  
    for eachline in fp:                                    #search for keywords and get linenumber
        m = re.search(keyword, eachline)
        if m is not None:
            number.append(lineNumber)                     #split by linenumber
        lineNumber+=1 
    number.append(len(open(INPUT).readlines()))
    number = list(set(number))
    number.sort()

    start = number[2]
    end =  number[3]
    middlelines = linecache.getlines(inputfile)[start:end-1]
    lines = []
    charge = []
    with open(newpath+outputfile+'.xyz','w') as fp_w:
        lines.append(str(len(middlelines))+'\n'+'\n')
        for i in range (len(middlelines)):
            # Split the line into individual values (assuming they are separated by spaces)
            values = middlelines[i].split()       
            # Extract values based on their positions in the format string
            #value1 = values[0] #index
            value2 = values[atom_label_column] #atom_label 
            value_atom = re.sub(r'\d', '', value2)
            #value2 = values[1] #atom_type
            value3 = float(values[2]) #x
            value4 = float(values[3]) #y
            value5 = float(values[4]) #z
            value6 = float(values[charge_column]) #charges
            charge.append(value6) 
     

            # Format the values using the specified format string
            formatted_line = "%-7s%-8.3f%-8.3f%-8.3f%-7s%-8.3f" % (
                value_atom, value3, value4, value5, value2, value6
            )
    
            lines.append(formatted_line+'\n')

        fp_w.writelines(lines)
    
    #print(sum(charge))
    return charge

def ff_gen(mae,grofile_name,residue_name):
    maefilestrip = mae.strip(".01.mae")
    newpath = os.path.abspath ( '')+'/'+str(maefilestrip)+'/' 
    xyzfile = newpath+maefilestrip+'.xyz'
#Import molecule and set molecule properties
    molecule = vlx.Molecule.read_xyz_file(xyzfile)
    '''no need to wake scf up or edit any lines below if you want to use resp from your *mae.01 file'''
    #molecule.set_charge(-4)  #TODO:
    #molecule.set_multiplicity(2) #TODO:
    #basis = vlx.MolecularBasis.read(molecule, 'sto-3g') #TODO:
#Set scf_driver
    #scf_drv = vlx.ScfUnrestrictedDriver() #TODO:
    #scf_drv.guess_unpaired_electrons = '60(1)' #TODO:
    #scf_drv.conv_thresh = 1e-1
    #scf_drv.max_iter = 500
    ##scf_drv.xcfun = 'b3lyp'
    #scf_results = scf_drv.compute(molecule, basis)
##Generate the force field with the scf results
    ff_gen = vlx.ForceFieldGenerator()
    ff_gen.ostream.mute()
    #ff_gen.create_topology(molecule, basis, scf_results)
    ff_gen.create_topology(molecule, no_resp=True)
    ff_gen.write_gromacs_files(grofile_name,residue_name)
    #molecule.show(atom_indices=True)
    
    #itp_files = glob.glob(os.path.join(path, '**/*.itp'), recursive=True)

    #os.rename($grofile_name*, destination_path) $grofile_name* $newpath
    mvff_files2subfolder(mae,grofile_name)

def get_resp_charge(esp_charge,equiv,std_thresh):
    equiv_atoms_groups = []

# Split the string by comma
    substrings = equiv.split(',')
    # Split each substring by "="
    for substr in substrings:
        unit = substr.split('=')
        equiv_atoms_groups.append(unit)

# Deal with RESP charge
    esp_charge.insert(0, 0.0)
    resp_charge = esp_charge
    stds = []
    for i in range(len(equiv_atoms_groups)):
        atoms = equiv_atoms_groups[i]
        #for every set of equivalent atoms charge, average them if difference is less than 0.01
        equiv_atoms_charges = []
        for j in range(len(atoms)):
            #print(atoms[j],charge[int(atoms[j])])
            equiv_atoms_charges.append(esp_charge[int(atoms[j])])
        arr = np.array(equiv_atoms_charges)
        std = np.std(arr)
        stds.append(std)

    check_large_std = any(value > std_thresh for value in stds)
    #print(stds)
    if check_large_std:
        print("Recalculate is needed for too large standard deviation")
        print(stds)
        return None
    else:
        print("execute average")
        for i in range(len(equiv_atoms_groups)):
            atoms = equiv_atoms_groups[i]
        #for every set of equivalent atoms charge, average them if difference is less than 0.01, get average charge
            equiv_atoms_charges = []
            for j in range(len(atoms)):
                #print(atoms[j],charge[int(atoms[j])])
                equiv_atoms_charges.append(float(esp_charge[int(atoms[j])]))
            mean_charge = sum(equiv_atoms_charges)/len(equiv_atoms_charges)
        #reassign the average charge to all equivalent atoms
            for k in range(len(atoms)):
                resp_charge[int(atoms[k])] = float(mean_charge)
        print("Total ESP:   "+str(sum(esp_charge)))
        return resp_charge

def get_equiv(mae):
    maefilestrip = mae.strip(".01.mae")
    newpath = os.path.abspath ( '')+'/'+str(maefilestrip)+'/' 
    xyzfile = newpath+maefilestrip+'.xyz'

    molecule = vlx.molecule._Molecule_read_xyz_file(xyzfile)
    id = vlx.AtomTypeIdentifier()
    id.ostream.mute()
    atom_type = id.generate_gaff_atomtypes(molecule) 
    #print(atom_type)
    id.identify_equivalences()
    equivalent_charges = id.equivalent_charges
    molecule.show(atom_indices=True)
    return equivalent_charges

def replace_resp(mae,resp_charge,grofile_name):
    subfolder = os.path.abspath ( '')+'/'+mae.strip(".01.mae")+'/' 
    itp_file_name= find_itp_file(subfolder,grofile_name)
    #print(resp_charge)
    if resp_charge is None:
        return "Recalcute! large standard deviation"
    else:
        #process resp and qtot
        if round(sum(resp_charge),1)!=0:
            print("unneutral system")
            coeff=round(sum(resp_charge),1)/sum(resp_charge)
            resp_charge=coeff*np.array(resp_charge)
        #for neutral system
        else:
            print("neutral system")
            resp_charge=np.array(resp_charge)
        qtot=[]
        for i in range(len(resp_charge)):
            qtot.append(sum(resp_charge[:i+1]))
        qtot = np.round(np.array(qtot),3)
        print("Total RESP:    "+str(sum(resp_charge)))
        #print(qtot[-1],len(qtot))

        #do replacement in itp
        with open(subfolder+itp_file_name,'r') as f:
            alllines = f.readlines()
            lineNumber = 1
            keyword = "atoms ]"                                   #input('Keyword:')
        for eachline in alllines:                                    #search for keywords and get linenumber
            m = re.search(keyword, eachline)
            if m is not None:
                start=lineNumber+1                  #split by linenumber
            lineNumber+=1 

        end =  start+len(resp_charge)-1
        head = alllines[0:start]
        
        middlelines = alllines[start:end]
        tail = alllines[end:]
        residue_name = head[-4].split()[0]

        print(middlelines[0])
        for i in range(len(middlelines)):
            part1 = middlelines[i][:21]
            part2 = middlelines[i][24:39]
            part3 = middlelines[i][50:69]
            part4 = middlelines[i][75:]
            middlelines[i]= part1+residue_name+part2+"%11.8f"%(resp_charge[i+1])+part3+"%6.3f"%(qtot[i+1])+part4
            #print(resp_charge[i+1],str(np.round(resp_charge[i+1],8)))
        
        with open(subfolder+'new'+itp_file_name,'w') as fp:
            fp.writelines(head)
            fp.writelines(middlelines)
            fp.writelines(tail)

        print(middlelines[0],middlelines[-1],itp_file_name+" update success")

def write_resp2ff(mae,charge_column,atom_label_column,std_thresh,grofile_name,residue_name):
    #step1 mae2xyz
    esp_charge=fileParse(mae,charge_column,atom_label_column)
    #step2 ff_gen() #will try how to only use GAFF
    ff_gen(mae,grofile_name,residue_name)
    #step3 replace charge
    esp_charge = fileParse(mae,charge_column,atom_label_column)
    equiv = get_equiv(mae)
    print(equiv)
    resp_charge = get_resp_charge(esp_charge,equiv,std_thresh)
    replace_resp(mae,resp_charge,grofile_name)
    return resp_charge

In [ ]:
mae = 'er_M1_Co2TCPP_opt_B3LYP-D3_LACVPss.01.mae'  # "*01.mae"
charge_column = 7 #default esp output column in *01.mae,count from 1 as first column
atom_label_column = 12
grofile_name = 'TCPP_lacvpss' 
residue_name = 'TCP' # 3letters
std_thresh = 0.12 #standard deviation threshold 

resp_charge= write_resp2ff(mae,charge_column,atom_label_column,std_thresh,grofile_name,residue_name)
